In [1]:
%load_ext autoreload
%autoreload now

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-08137aa2-e69b-5e74-8390-7997329b1336"
# os.environ["WORLD_SIZE"] = "1"

# Download and convert data

In [3]:
from tqdm import tqdm

from document_segmentation.pagexml.generale_missiven import GeneraleMissiven
from document_segmentation.settings import (
    GENERALE_MISSIVEN_DOCUMENT_DIR,
    GENERALE_MISSIVEN_SHEET,
)

N = None

GENERALE_MISSIVEN_DOCUMENT_DIR.mkdir(parents=True, exist_ok=True)

sheet = GeneraleMissiven(GENERALE_MISSIVEN_SHEET)

existing_docs = {
    path.stem
    for path in GENERALE_MISSIVEN_DOCUMENT_DIR.glob("*.json")
    if path.is_file()
}

for document in tqdm(
    sheet.to_documents(n=N, skip_ids=existing_docs),
    total=(N or len(sheet)) - len(existing_docs),
    desc="Writing documents",
    unit="doc",
):
    document_file = GENERALE_MISSIVEN_DOCUMENT_DIR / f"{document.id}.json"

    with document_file.open("xt") as f:
        f.write(document.model_dump_json())
        f.write("\n")

Writing documents:   0%|          | 0/5 [00:00<?, ?doc/s]

Skipping row with inventory number 1171 due to status message: 'Niet gedigitaliseerd.'
Skipping row with inventory number 2770 due to status message: 'Niet gedigitaliseerd.'
Skipping row with inventory number 2770 due to status message: 'Niet gedigitaliseerd.'
Skipping row with inventory number 2770 due to status message: 'Niet gedigitaliseerd.'
Skipping row with inventory number 2911 due to status message: 'Niet gedigitaliseerd.'


# Load Data

In [4]:
%autoreload now

In [5]:
TRAINING_DATA = 0.8

In [6]:
from document_segmentation.model.dataset import PageDataset
from document_segmentation.settings import MIN_REGION_TEXT_LENGTH

dataset = PageDataset.from_dir(
    GENERALE_MISSIVEN_DOCUMENT_DIR, n=100
).remove_short_regions(MIN_REGION_TEXT_LENGTH)
len(dataset)

Reading JSON files: 100%|██████████| 100/100 [00:07<00:00, 13.43file/s]


20750

In [7]:
dataset._class_counts()

Counter({<Label.IN: 2>: 20552, <Label.BEGIN: 1>: 100, <Label.END: 3>: 98})

In [8]:
dataset.class_weights()

[205.44554455445544, 1.0095849754293778, 209.59595959595958, 20750.0]

In [9]:
dataset[5000]

Page(label=<Label.IN: 2>, regions=[Region(id='region_9df849b1-56aa-48d5-b056-961b95080a78_1', types=(<RegionType.PARAGRAPH: 'paragraph'>, <RegionType.PHYSICAL_STRUCTURE_DOC: 'physical_structure_doc'>, <RegionType.TEXT_REGION: 'text_region'>, <RegionType.PAGEXML_DOC: 'pagexml_doc'>), coordinates=((927, 671), (946, 693), (949, 715), (1000, 725), (1044, 747), (1060, 766), (1047, 835), (990, 867), (930, 921), (937, 1054), (930, 1180), (943, 1221), (889, 1395), (892, 1417), (914, 1458), (895, 1528), (908, 1658), (902, 1750), (905, 1892), (883, 1974), (886, 2015), (876, 2053), (883, 2085), (867, 2155), (823, 2237), (788, 2265), (791, 2306), (807, 2329), (807, 2360), (845, 2385), (851, 2408), (848, 2461), (927, 2544), (921, 2563), (902, 2572), (873, 2604), (851, 2673), (854, 2705), (873, 2733), (876, 2851), (899, 2961), (902, 3123), (918, 3167), (918, 3192), (864, 3293), (870, 3331), (911, 3357), (952, 3366), (1206, 3366), (1338, 3379), (1598, 3376), (1772, 3385), (1971, 3376), (2091, 3392), 

In [10]:
split = int(len(dataset) * TRAINING_DATA)

training_data = dataset[:split]
training_data._class_counts()

Counter({<Label.IN: 2>: 16456, <Label.BEGIN: 1>: 73, <Label.END: 3>: 71})

In [11]:
test_data = dataset[split:]
test_data._class_counts()

Counter({<Label.IN: 2>: 4096, <Label.END: 3>: 27, <Label.BEGIN: 1>: 27})

# Train Model

In [12]:
import torch

BATCH_SIZE = 16
EPOCHS = 3
WEIGHTS = torch.Tensor(dataset.class_weights())  # For an imbalanced dataset

In [43]:
%autoreload now

In [44]:
from document_segmentation.model.page_sequence_tagger import PageSequenceTagger

tagger = PageSequenceTagger()

In [45]:
tagger._device

'mps'

In [46]:
tagger

PageSequenceTagger(
  (_page_embedding): PageEmbedding(
    (_region_model): RegionEmbedding(
      (_transformer_model): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(42774, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1, inplace=False)
     

In [47]:
tagger.train_(training_data, EPOCHS, BATCH_SIZE, WEIGHTS)

  0%|          | 0/1037.5 [00:01<?, ?batch/s]


RuntimeError: Given groups=1, weight of size [1, 100, 3, 3], expected input[1, 16, 4, 4] to have 100 channels, but got 16 channels instead

# Evaluate Model

In [ ]:
import csv
import sys

from torcheval.metrics import (
    MulticlassAccuracy,
    MulticlassF1Score,
    MulticlassPrecision,
    MulticlassRecall,
)
from tqdm import tqdm

from document_segmentation.pagexml.datamodel.label import Label

writer = csv.DictWriter(
    sys.stdout,
    fieldnames=("Predicted", "Actual", "Page ID", "Text", "Scores"),
    delimiter="\t",
)

writer.writeheader()

accuracy = MulticlassAccuracy(num_classes=len(Label))
precision = MulticlassPrecision(average=None, num_classes=len(Label))
recall = MulticlassRecall(average=None, num_classes=len(Label))
f1_score = MulticlassF1Score(average=None, num_classes=len(Label))

for batch in tqdm(
    test_data.batches(BATCH_SIZE), total=len(test_data) / BATCH_SIZE, unit="batch"
):
    predicted = tagger(batch)
    labels = batch.labels()

    _labels = torch.Tensor([label.value - 1 for label in labels]).to(int)
    accuracy.update(predicted, _labels)
    precision.update(predicted, _labels)
    recall.update(predicted, _labels)
    f1_score.update(predicted, _labels)

    for page, pred, label in zip(batch.pages, predicted, labels):
        pred_label = Label(pred.argmax().item() + 1)
        if pred_label != Label.IN or label != Label.IN:
            writer.writerow(
                {
                    "Predicted": pred_label.name,
                    "Actual": label.name,
                    "Page ID": page.doc_id,
                    "Text": page.text(delimiter="; ")[:50],
                    "Scores": str(pred.tolist()),
                }
            )

Predicted	Actual	Page ID	Text	Scores


  2%|▏         | 3/129.6875 [00:00<00:05, 22.72batch/s]

BEGIN	IN	NL-HaNA_1.04.02_3533_0317.jpg	zouden doen dragen, dat geene dubbelde; dukatons z	[0.9711317420005798, 0.028105031698942184, 0.00037071146653033793, 0.00039246713276952505]
BEGIN	IN	NL-HaNA_1.04.02_3533_0318.jpg	aanbesteede; 'sComp„s reparatie tot Th: 100:; mede	[0.6227352619171143, 0.37679725885391235, 0.00024215613666456193, 0.00022533860465046018]
BEGIN	IN	NL-HaNA_1.04.02_3533_0349.jpg	worden gelijd, dat die frifatie/ bedongen inkoop; 	[0.9723415374755859, 0.026956947520375252, 0.00034019837039522827, 0.00036138645373284817]
BEGIN	IN	NL-HaNA_1.04.02_3533_0350.jpg	begunstigde Britsche vrienden van ons; afkeerig is	[0.598449170589447, 0.40105879306793213, 0.00025473529240116477, 0.00023724768834654242]
BEGIN	IN	NL-HaNA_1.04.02_3533_0381.jpg	„meld onder aanhaaling, dat de noodsaake„; lijkhei	[0.9586215019226074, 0.04012445732951164, 0.0006167643587104976, 0.0006373219075612724]
BEGIN	IN	NL-HaNA_1.04.02_3533_0382.jpg	dat de Comt. e en de Colonie ginter volgens der; m	[0.69552469

  7%|▋         | 9/129.6875 [00:00<00:05, 23.66batch/s]

BEGIN	IN	NL-HaNA_1.04.02_3533_0509.jpg	Dat particuliere Engelsen alleen met; sal peter na	[0.9645834565162659, 0.034492310136556625, 0.00045255740405991673, 0.00047164104762487113]
BEGIN	IN	NL-HaNA_1.04.02_3533_0510.jpg	7dd; „o of ƒ 13. 17 2/5. , dan bleek het schie„; l	[0.556537389755249, 0.44290244579315186, 0.0002930821792688221, 0.0002670731919351965]
BEGIN	IN	NL-HaNA_1.04.02_3533_0541.jpg	van'teen en ander word verder gehandeld in; de bri	[0.9697939157485962, 0.029392404481768608, 0.00039518537232652307, 0.0004184711433481425]
BEGIN	IN	NL-HaNA_1.04.02_3533_0542.jpg	missive van den 8:' october 1777. § 300:/ En; ton 	[0.5456979274749756, 0.453619122505188, 0.00035544708953239024, 0.000327510351780802]
BEGIN	IN	NL-HaNA_1.04.02_3533_0573.jpg	Een somma van 800 ropijen sijnde ƒ 1000. boven; de	[0.9778214693069458, 0.02146643027663231, 0.0003412238438613713, 0.00037098146276548505]
BEGIN	IN	NL-HaNA_1.04.02_3533_0574.jpg	elkander te krijgen was, dienen moest voor; het fo	[0.57876169681549

...

 36%|███▌      | 47/129.6875 [00:34<03:10,  2.31s/batch]

BEGIN	IN	NL-HaNA_1.04.02_2140_0147.jpg	al het geene de ministers op; den; Eijsch van uEd:	[0.9437512755393982, 0.053762540221214294, 0.0012183841317892075, 0.0012677614577114582]
BEGIN	IN	NL-HaNA_1.04.02_2140_0148.jpg	de scheepen; wickenburg en; is vorderde goederen h	[0.6372575163841248, 0.36147570610046387, 0.0006543862400576472, 0.0006124170613475144]


 37%|███▋      | 48/129.6875 [00:37<01:03,  1.29batch/s]

BEGIN	IN	NL-HaNA_1.04.02_2140_0179.jpg	die nergens toe kan werden; g' Emploijeert maar oo	[0.9455181956291199, 0.052035294473171234, 0.0011997788678854704, 0.001246739993803203]
BEGIN	IN	NL-HaNA_1.04.02_2140_0180.jpg	overgaan Een monster, ter qu; titeijt van 37½ lb:,	[0.8264032602310181, 0.1730942577123642, 0.0002529376943130046, 0.0002495023945812136]


KeyboardInterrupt: 

In [ ]:
writer = csv.DictWriter(
    sys.stdout,
    fieldnames=["Metric", "Average"] + [label.name for label in Label],
    delimiter="\t",
)
writer.writeheader()

for metric in (precision, recall, f1_score):
    scores = {
        label.name: f"{score:.4f}"
        for label, score in zip(Label, metric.compute().tolist())
    }
    writer.writerow(
        {"Metric": metric.__class__.__name__, "Average": str(metric.average)} | scores
    )

print(f"Accuracy ({accuracy.average} average):\t{accuracy.compute().item():.4f}")

        [3]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.


Metric	Average	BEGIN	IN	END	OUT
MulticlassPrecision	None	0.0000	1.0000	0.0000	0.0000
MulticlassRecall	None	0.0000	0.9300	0.0000	0.0000
MulticlassF1Score	None	0.0000	0.9637	0.0000	0.0000
Accuracy (micro average):	0.9300
